In [1]:
import pandas as pd
from pathlib import Path
from fiona.crs import from_epsg
from fiona import open as fopen
from shapely.geometry import Point, LineString
import ast

In [2]:
porto_csv_path = Path('./data/sources/porto/train.csv')
roma_csv_path = Path('./data/sources/roma/taxi_february.txt')
df = pd.read_csv(porto_csv_path)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [ ]:
min_two_points_mask = df["POLYLINE"].str.count(",") > 1
point_filter_df = df[min_two_points_mask]

In [ ]:
df.size - point_filter_df.size

328590

In [ ]:
schema = {
    'geometry': 'LineString',
    'properties': {
        'trip_id': 'int',
        'call_type': 'str:80',
        # 'origin_call': 'float:24.5',
        # 'origin_stand': 'float:24.5',
        'taxi_id': 'int',
        'timestamp': 'int',
        'day_type': 'str:80',
        'missing_data': 'int',
    }
}

porto_output_path = Path('./data/generated/train').resolve()

def append_to_shp(row, shp):
    polyline = ast.literal_eval(row['POLYLINE'])
    line_string = LineString(polyline)
    shp.write({
        'geometry': {
            'type': 'LineString',
            'coordinates': line_string.coords,
        },
        'properties': {
            'trip_id': row['TRIP_ID'],
            'call_type': row['CALL_TYPE'],
            # 'origin_call': row['ORIGINAL_CALL'],
            # 'origin_stand': row['ORIGIN_STAND'],
            'taxi_id': row['TAXI_ID'],
            'timestamp': row['TIMESTAMP'],
            'day_type': row['DAY_TYPE'],
            'missing_data': row['MISSING_DATA'],
        }
    })  
    

In [ ]:
with fopen(porto_output_path, 'w', driver='ESRI Shapefile', crs=from_epsg(4326), schema=schema) as shp:
    point_filter_df.apply(append_to_shp, shp=shp, axis=1)

KeyboardInterrupt: 